# TD1 — Modèle en étoile (SQLite)

## Objectifs

- Construire schéma étoile.
- Insérer données.
- Requêtes validation.

In [ ]:
# Import SQLite
import sqlite3
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
print('Base TD1 créée')

## DDL des tables

In [ ]:
# DDL dimensions et faits
cursor.execute('''
CREATE TABLE dim_date (
  date_id INTEGER PRIMARY KEY AUTOINCREMENT,
  date_cal DATE NOT NULL,
  annee INTEGER,
  mois INTEGER,
  jour INTEGER,
  trimestre INTEGER
);
''')

cursor.execute('''
CREATE TABLE dim_produit (
  produit_sk INTEGER PRIMARY KEY AUTOINCREMENT,
  produit_id TEXT UNIQUE,
  produit_nom TEXT,
  categorie TEXT
);
''')

cursor.execute('''
CREATE TABLE dim_magasin (
  magasin_sk INTEGER PRIMARY KEY AUTOINCREMENT,
  magasin_id TEXT UNIQUE,
  ville TEXT,
  region TEXT
);
''')

cursor.execute('''
CREATE TABLE fact_ventes (
  fact_sk INTEGER PRIMARY KEY AUTOINCREMENT,
  date_id INTEGER NOT NULL,
  produit_sk INTEGER NOT NULL,
  magasin_sk INTEGER NOT NULL,
  montant REAL,
  quantite INTEGER,
  FOREIGN KEY (date_id) REFERENCES dim_date(date_id),
  FOREIGN KEY (produit_sk) REFERENCES dim_produit(produit_sk),
  FOREIGN KEY (magasin_sk) REFERENCES dim_magasin(magasin_sk)
);
''')
print('Tables créées')

## Inserts d'exemple

In [ ]:
# Inserts dimensions
cursor.executemany('''
INSERT INTO dim_date (date_cal, annee, mois, jour, trimestre) VALUES (?, ?, ?, ?, ?);
''', [
    ('2024-01-02', 2024, 1, 2, 1),
    ('2024-01-03', 2024, 1, 3, 1),
    ('2024-01-04', 2024, 1, 4, 1)
])

cursor.executemany('''
INSERT INTO dim_produit (produit_id, produit_nom, categorie) VALUES (?, ?, ?);
''', [
    ('P01', 'Chemise Oxford', 'Textile'),
    ('P02', 'Sneakers Run', 'Chaussure'),
    ('P03', 'Jeans Slim', 'Textile')
])

cursor.executemany('''
INSERT INTO dim_magasin (magasin_id, ville, region) VALUES (?, ?, ?);
''', [
    ('M01', 'Paris', 'IDF'),
    ('M02', 'Lyon', 'ARA')
])

# Inserts faits
cursor.executemany('''
INSERT INTO fact_ventes (date_id, produit_sk, magasin_sk, montant, quantite) VALUES (?, ?, ?, ?, ?);
''', [
    (1, 1, 1, 120.0, 2),
    (1, 2, 1, 75.0, 1),
    (2, 1, 2, 60.0, 1),
    (3, 3, 1, 90.0, 1)
])
print('Données insérées')

## Requêtes de validation

In [ ]:
# Volume total
cursor.execute('SELECT COUNT(*) AS nb_lignes FROM fact_ventes;')
print('Volume total :', cursor.fetchone()[0])

# Top 3 produits par CA
cursor.execute('''
SELECT p.produit_nom, SUM(f.montant) AS ca
FROM fact_ventes f
JOIN dim_produit p ON f.produit_sk = p.produit_sk
GROUP BY p.produit_nom
ORDER BY ca DESC
LIMIT 3;
''')
print('Top 3 produits :')
for row in cursor.fetchall():
    print(row)

# CA par ville et par mois
cursor.execute('''
SELECT d.annee, d.mois, m.ville, SUM(f.montant) AS ca
FROM fact_ventes f
JOIN dim_date d ON f.date_id = d.date_id
JOIN dim_magasin m ON f.magasin_sk = m.magasin_sk
GROUP BY d.annee, d.mois, m.ville
ORDER BY d.annee, d.mois, m.ville;
''')
print('CA par ville/mois :')
for row in cursor.fetchall():
    print(row)

conn.close()
print('TD1 terminé')